In [ ]:
import pandas as pd
import json
from datasets import Dataset

# Загружаем готовые answers.csv файлы (сгенерированные через generate_answers.py)
print("Загрузка answers.csv файлов...")

# БЕЗ реранкера
df_no_rerank = pd.read_csv('answers_no_rerank.csv')
print(f"✓ Загружено {len(df_no_rerank)} записей БЕЗ реранкера")

# С реранкером
df_with_rerank = pd.read_csv('answers_with_rerank.csv')
print(f"✓ Загружено {len(df_with_rerank)} записей С реранкером")

# Показываем первые строки
print("\nПервые строки БЕЗ реранкера:")
df_no_rerank.head()

FileNotFoundError: [Errno 2] No such file or directory: 'answers.csv'

In [ ]:
# Подготовка датасетов для RAGAS из готовых CSV файлов
print("\nПодготовка датасетов для RAGAS...")

# Функция для преобразования CSV в Dataset для RAGAS
def prepare_dataset_from_csv(df):
    """Преобразует DataFrame в Dataset для RAGAS."""
    # Преобразуем context из JSON строки в список
    df['context'] = df['context'].str.replace("'", '"', regex=False)
    
    data_samples = {
        'question': df['query'].tolist(),
        'answer': df['answer'].tolist(),
        'contexts': [json.loads(x) for x in df['context'].tolist()],
        'ground_truth': df['gold_answers'].tolist()
    }
    
    return Dataset.from_dict(data_samples)

# Создаем датасеты
dataset_no_rerank = prepare_dataset_from_csv(df_no_rerank)
dataset_with_rerank = prepare_dataset_from_csv(df_with_rerank)

print(f"✓ Подготовлено {len(dataset_no_rerank)} записей без реранкера")
print(f"✓ Подготовлено {len(dataset_with_rerank)} записей с реранкером")


In [ ]:
# Импорт RAGAS метрик (актуальная версия без _ru)
import os
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings import GigaChatEmbeddings
from ragas import evaluate, RunConfig
from dotenv import load_dotenv

load_dotenv()

# Пытаемся импортировать из нового API
try:
    from ragas.metrics.collections import (
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_utilization
    )
    metrics_to_evaluate = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_utilization
    ]
    print("✓ Метрики загружены из ragas.metrics.collections")
except ImportError:
    # Fallback для старых версий
    from ragas.metrics import (
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    )
    metrics_to_evaluate = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
    try:
        from ragas.metrics import context_utilization
        metrics_to_evaluate.append(context_utilization)
    except ImportError:
        print("⚠ context_utilization недоступен")
    print("✓ Метрики загружены из ragas.metrics")

# Настройка LLM и embeddings для RAGAS
ragas_llm = GigaChat(
    credentials=os.getenv("GIGACHAT_AUTH_KEY"),
    scope="GIGACHAT_API_PERS",
    model='GigaChat-2',
    verify_ssl_certs=False,
    temperature=0.0
)

ragas_embeddings = GigaChatEmbeddings(
    credentials=os.getenv("GIGACHAT_AUTH_KEY"),
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False
)

def on_error(e, job):
    print(f"ERROR in job {job}: {e}")

run_config = RunConfig(
    max_workers=1,
    callbacks={"on_error": on_error}
)

print("\n" + "="*60)
print("ВЫЧИСЛЕНИЕ МЕТРИК RAGAS")
print("="*60)
print(f"Используемые метрики: {[m.name for m in metrics_to_evaluate if m is not None]}")

In [ ]:
# Импорт RAGAS метрик (актуальная версия без _ru)
from langchain_gigachat.embeddings import GigaChatEmbeddings
from ragas import evaluate, RunConfig

# Пытаемся импортировать из нового API
try:
    from ragas.metrics.collections import (
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_utilization
    )
    metrics_to_evaluate = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_utilization
    ]
    print("✓ Метрики загружены из ragas.metrics.collections")
except ImportError:
    # Fallback для старых версий
    from ragas.metrics import (
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    )
    metrics_to_evaluate = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
    try:
        from ragas.metrics import context_utilization
        metrics_to_evaluate.append(context_utilization)
    except ImportError:
        print("⚠ context_utilization недоступен")
    print("✓ Метрики загружены из ragas.metrics")

# Настройка LLM и embeddings для RAGAS
ragas_llm = GigaChat(
    credentials=os.getenv("GIGACHAT_AUTH_KEY"),
    scope="GIGACHAT_API_PERS",
    model='GigaChat-2',
    verify_ssl_certs=False,
    temperature=0.0
)

ragas_embeddings = GigaChatEmbeddings(
    credentials=os.getenv("GIGACHAT_AUTH_KEY"),
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False
)

def on_error(e, job):
    print(f"ERROR in job {job}: {e}")

run_config = RunConfig(
    max_workers=1,
    callbacks={"on_error": on_error}
)

print("\n" + "="*60)
print("ВЫЧИСЛЕНИЕ МЕТРИК RAGAS")
print("="*60)
print(f"Используемые метрики: {[m.name for m in metrics_to_evaluate if m is not None]}")

⚠ context_utilization недоступен
✓ Метрики загружены из ragas.metrics


/var/folders/s5/wgbcfr7944sb1lf88j5m_3ld4qjxdj/T/ipykernel_34515/951723274.py:25: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/s5/wgbcfr7944sb1lf88j5m_3ld4qjxdj/T/ipykernel_34515/951723274.py:25: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/s5/wgbcfr7944sb1lf88j5m_3ld4qjxdj/T/ipykernel_34515/951723274.py:25: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/

TypeError: RunConfig.__init__() got an unexpected keyword argument 'callbacks'

In [ ]:
# Оценка БЕЗ реранкера
print("\n1. Оценка БЕЗ реранкера...")
score_no_rerank = evaluate(
    dataset=dataset_no_rerank,
    metrics=metrics_to_evaluate,
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    run_config=run_config
)

df_no_rerank = score_no_rerank.to_pandas()
print("✓ Метрики без реранкера вычислены")
df_no_rerank.head()
